In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df

In [ ]:
import re 
def review_cleaner(reviews):
    pattern = r'[^\w\s]'
    reviews = re.sub(pattern, '', reviews)
    reviews = re.sub('✅', '', reviews) # remove checkmark emojis
    reviews = re.sub('❌', '', reviews) # remove X emojis
    reviews = re.sub('Trip Verified', '', reviews) # remove Trip Verified
    reviews = re.sub('Not Verified\s*\|?\s*', '', reviews) # remove Not Verified
    reviews = re.sub('Verified Review', '', reviews)
    reviews = reviews.replace('|', '') # Remove Vertical Bar
    reviews = reviews.lower() # Lowercase the text
    return reviews

df['clean_reviews'] = df['Reviews'].apply(review_cleaner)
df

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess(reviews):
    stop_words = set(stopwords.words('english'))  # Load English stopwords from NLTK
    words = reviews.split()  # Tokenize text into words
    filtered_words = [word for word in words if word.lower() not in stop_words]  # Remove stopwords
    return ' '.join(filtered_words)

df['clean_reviews2'] = df['clean_reviews'].apply(preprocess)
df

In [ ]:
def df_split(df):
    num_dfs = len(df) // 500 + 1
    dfs = []
    for i in range(num_dfs):
        start_idx = i * 500
        end_idx = min((i + 1) * 500, len(df))
        var_name = f"df{i}"  # Create a variable name
        split_df = pd.DataFrame(df[start_idx:end_idx])
        # Store the DataFrame using the variable name
        globals()[var_name] = split_df
        dfs.append(split_df)  # Append the DataFrame to the list
        
    return dfs

# Example usage:
dfs = df_split(df)
print(dfs)

In [ ]:
from transformers import pipeline
sentiment_analysis = pipeline("text-classification", model="finiteautomata/bertweet-base-sentiment-analysis")

In [ ]:
def get_sentiment(text):
    try:
        sentiment = sentiment_analysis(text[:512])
        return sentiment[0]['label']
    except:
        return "Error"

In [ ]:
for som in dfs:
    som['sentiment'] = som['clean_reviews2'].apply(get_sentiment)

In [ ]:
concatenated_df = pd.concat(dfs, ignore_index=True)
concatenated_df

In [ ]:
df_filtered = concatenated_df[~concatenated_df['sentiment'].str.contains('Error', na=False)]
df_filtered

In [ ]:
concatenated_df.to_csv('alaska-airlines/Alaska_Reviews_Clean.csv', index = False)